In [1]:
import plotly.express as px
import pandas as pd
import json
import os
os.environ['USER'] = "rmoine"
from pathlib import Path
import numpy as np
from typing import *
from llama.main import compute_metrics_from_files, DatasetName, compute_metrics_from_list
from PIL import Image
from io import BytesIO
import base64
from plotly.graph_objects import Figure
import win32clipboard
import textwrap
import colorsys
def img_to_clipboard(fig: Figure):
    img_bytes = fig.to_image(format="png")
    image = Image.open(BytesIO(img_bytes))
    
    output = BytesIO()
    image.convert('RGB').save(output, 'BMP')
    data = output.getvalue()[14:]
    output.close()
    win32clipboard.OpenClipboard()
    win32clipboard.EmptyClipboard()
    win32clipboard.SetClipboardData(win32clipboard.CF_DIB, data)
    win32clipboard.CloseClipboard()

c:\Users\robin\miniconda3\envs\severityPrediction\lib\site-packages\bitsandbytes\cextension.py:34: UserWarning: The installed version of bitsandbytes was compiled without GPU support. 8-bit optimizers, 8-bit multiplication, and GPU quantization are unavailable.
  warn("The installed version of bitsandbytes was compiled without GPU support. "


'NoneType' object has no attribute 'cadam32bit_grad_fp32'
Import of import evaluate failed
Import of from pytorchtools import EarlyStopping failed


In [2]:
l = []
l1 = []
root = Path(f"../../data/finetuning/")
i_tot = 0
for path_data in root.rglob("qlora_finetune_*"):
    print(path_data)
    with open(path_data / "parameters.json") as fp:
        parameters = json.load(fp)
    for dataset_type in ["train","val"]:
        files_data = list(path_data.rglob(f"data_epoch_*_{dataset_type}*.json"))
        if len(files_data) == 0:
            continue
        i_tot += 1
        i = i_tot//2
        for f in files_data:
            with open(f) as fp:
                d = json.load(fp)
            confusion_matrix, f1, _ = compute_metrics_from_list(d, pred_field="prediction")
            accuracy = np.sum(np.diag(confusion_matrix)) / np.sum(confusion_matrix)
            epoch = int(f.stem.split("_")[-2].split("-")[0])
            loss = sum(e['loss'] for e in d)/len(d)
            for e in d:
                l1.append({"epoch":epoch,"loss":e['loss'],"dataset_type":dataset_type, "i":i, "path_data_folder": path_data.stem, **parameters})
            l.append({"epoch":epoch,"dataset_type":dataset_type,"confusion_matrix":confusion_matrix,"accuracy":accuracy, "path_data_folder": path_data.stem, "loss": loss, "i":i, **parameters})
df = pd.DataFrame(l)
display(df.head())


..\..\data\finetuning\qlora_finetune_19346995
..\..\data\finetuning\qlora_finetune_19346996
..\..\data\finetuning\qlora_finetune_19347000
..\..\data\finetuning\qlora_finetune_19347001
..\..\data\finetuning\qlora_finetune_19348539
..\..\data\finetuning\qlora_finetune_19348541
..\..\data\finetuning\qlora_finetune_19348542
..\..\data\finetuning\qlora_finetune_19348543
..\..\data\finetuning\qlora_finetune_19348552
..\..\data\finetuning\qlora_finetune_19348553
..\..\data\finetuning\qlora_finetune_eclipse_72k
..\..\data\finetuning\qlora_finetune_mozilla_200k
..\..\data\finetuning\qlora_finetune_weighted_eclipse_72k
..\..\data\finetuning\qlora_finetune_weighted_mozilla_200k


,epoch,dataset_type,confusion_matrix,accuracy,path_data_folder,loss,i,dataset_choice,folder_out,folder_data,...,num_train_epochs,tr_bs,gradient_accumulation_steps,learning_rate,limit_tokens,mapping_dict,lim_size,id,use_cpu,tr_weighted_sampling
0,0,train,"[[9268, 3396], [3540, 9124]]",0.726153,qlora_finetune_19348539,0.609922,0,eclipse_72k,/project/6023391/rmoine/data,/project/6023391/rmoine/data,...,500,4,1,0.0001,500,None,-1,_19348539,False,True
1,1,train,"[[9971, 2693], [2713, 9951]]",0.786560,qlora_finetune_19348539,0.544867,0,eclipse_72k,/project/6023391/rmoine/data,/project/6023391/rmoine/data,...,500,4,1,0.0001,500,None,-1,_19348539,False,True
2,2,train,"[[10318, 2346], [2324, 10340]]",0.815619,qlora_finetune_19348539,0.514827,0,eclipse_72k,/project/6023391/rmoine/data,/project/6023391/rmoine/data,...,500,4,1,0.0001,500,None,-1,_19348539,False,True
3,3,train,"[[10775, 1889], [2051, 10613]]",0.844441,qlora_finetune_19348539,0.480139,0,eclipse_72k,/project/6023391/rmoine/data,/project/6023391/rmoine/data,...,500,4,1,0.0001,500,None,-1,_19348539,False,True
4,4,train,"[[11273, 1391], [1727, 10937]]",0.876895,qlora_finetune_19348539,0.429081,0,eclipse_72k,/project/6023391/rmoine/data,/project/6023391/rmoine/data,...,500,4,1,0.0001,500,None,-1,_19348539,False,True


In [3]:
print(df.columns)
df.sort_values(["dataset_choice","dataset_type","epoch"],inplace=True)
df["weighted"] = df["tr_weighted_sampling"].apply(lambda x:"weighted" if x else "not_weighted")
df["accuracy_prct"] = df["accuracy"]*100
df["name"] = df['i'].astype(str) + " " + df["dataset_choice"] + ", " + df["weighted"] + ", r=" + df["lora_r"].astype(str)
df["id"] = df["name"] + ", " + df["dataset_type"]
df.sort_values(by=["i","dataset_type","epoch"],inplace=True)
display(df)

Index(['epoch', 'dataset_type', 'confusion_matrix', 'accuracy',
       'path_data_folder', 'loss', 'i', 'dataset_choice', 'folder_out',
       'folder_data', 'lora_alpha', 'lora_dropout', 'lora_r', 'model_name',
       'token', 'field_input', 'num_train_epochs', 'tr_bs',
       'gradient_accumulation_steps', 'learning_rate', 'limit_tokens',
       'mapping_dict', 'lim_size', 'id', 'use_cpu', 'tr_weighted_sampling'],
      dtype='object')


,epoch,dataset_type,confusion_matrix,accuracy,path_data_folder,loss,i,dataset_choice,folder_out,folder_data,...,learning_rate,limit_tokens,mapping_dict,lim_size,id,use_cpu,tr_weighted_sampling,weighted,accuracy_prct,name
0,0,train,"[[9268, 3396], [3540, 9124]]",0.726153,qlora_finetune_19348539,0.609922,0,eclipse_72k,/project/6023391/rmoine/data,/project/6023391/rmoine/data,...,0.0001,500,None,-1,"0 eclipse_72k, weighted, r=10, train",False,True,weighted,72.615287,"0 eclipse_72k, weighted, r=10"
1,1,train,"[[9971, 2693], [2713, 9951]]",0.786560,qlora_finetune_19348539,0.544867,0,eclipse_72k,/project/6023391/rmoine/data,/project/6023391/rmoine/data,...,0.0001,500,None,-1,"0 eclipse_72k, weighted, r=10, train",False,True,weighted,78.656033,"0 eclipse_72k, weighted, r=10"
2,2,train,"[[10318, 2346], [2324, 10340]]",0.815619,qlora_finetune_19348539,0.514827,0,eclipse_72k,/project/6023391/rmoine/data,/project/6023391/rmoine/data,...,0.0001,500,None,-1,"0 eclipse_72k, weighted, r=10, train",False,True,weighted,81.561908,"0 eclipse_72k, weighted, r=10"
3,3,train,"[[10775, 1889], [2051, 10613]]",0.844441,qlora_finetune_19348539,0.480139,0,eclipse_72k,/project/6023391/rmoine/data,/project/6023391/rmoine/data,...,0.0001,500,None,-1,"0 eclipse_72k, weighted, r=10, train",False,True,weighted,84.444093,"0 eclipse_72k, weighted, r=10"
4,4,train,"[[11273, 1391], [1727, 10937]]",0.876895,qlora_finetune_19348539,0.429081,0,eclipse_72k,/project/6023391/rmoine/data,/project/6023391/rmoine/data,...,0.0001,500,None,-1,"0 eclipse_72k, weighted, r=10, train",False,True,weighted,87.689514,"0 eclipse_72k, weighted, r=10"
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
93,2,val,"[[3017, 601], [2668, 8152]]",0.773584,qlora_finetune_weighted_eclipse_72k,0.555981,9,eclipse_72k,/project/6023391/rmoine/data,/project/6023391/rmoine/data,...,0.0001,500,None,-1,"9 eclipse_72k, weighted, r=64, val",False,True,weighted,77.358360,"9 eclipse_72k, weighted, r=64"
94,3,val,"[[3086, 532], [2821, 7999]]",0.767766,qlora_finetune_weighted_eclipse_72k,0.628481,9,eclipse_72k,/project/6023391/rmoine/data,/project/6023391/rmoine/data,...,0.0001,500,None,-1,"9 eclipse_72k, weighted, r=64, val",False,True,weighted,76.776562,"9 eclipse_72k, weighted, r=64"
95,4,val,"[[2765, 853], [1717, 9103]]",0.821998,qlora_finetune_weighted_eclipse_72k,0.618397,9,eclipse_72k,/project/6023391/rmoine/data,/project/6023391/rmoine/data,...,0.0001,500,None,-1,"9 eclipse_72k, weighted, r=64, val",False,True,weighted,82.199751,"9 eclipse_72k, weighted, r=64"
98,1,val,"[[7664, 2857], [4888, 24756]]",0.807170,qlora_finetune_weighted_mozilla_200k,0.426377,10,mozilla_200k,/project/6023391/rmoine/data,/project/6023391/rmoine/data,...,0.0001,500,None,-1,"10 mozilla_200k, weighted, r=64, val",False,True,weighted,80.717042,"10 mozilla_200k, weighted, r=64"


In [4]:
df[["i","path_data_folder"]].drop_duplicates()

,i,path_data_folder
0,0,qlora_finetune_19348539
14,1,qlora_finetune_19348541
7,1,qlora_finetune_19348539
21,2,qlora_finetune_19348542
18,2,qlora_finetune_19348541
28,3,qlora_finetune_19348543
25,3,qlora_finetune_19348542
35,4,qlora_finetune_19348552
32,4,qlora_finetune_19348543
42,5,qlora_finetune_19348553


In [105]:
l = df.iloc[0]
width, height = 800, 600
title_attrs = {
    "bs":l.tr_bs,
    "lora_alpha":l.lora_alpha,
    "lora_dropout":l.lora_dropout,
    "model_name":l.model_name.split('/')[1],
    "learning_rate": l.learning_rate,
    
}
unique_datasets = list(df["name"].unique())[::2]
print(unique_datasets)
title = "<br>".join(textwrap.wrap(" ; ".join(f"{k}={v}" for k,v in title_attrs.items()), width=75))
color_map = {}
for i, dataset in enumerate(sorted(unique_datasets)):
    base_color = [int(i*360/len(unique_datasets)),1,1]
    i = int(dataset.split(" ")[0])
    dataset = " ".join(dataset.split(" ")[1:])
    color_map[f"{i} {dataset}, train"] = f'hsv({base_color[0]}, 1, 1)'
    color_map[f"{i+1} {dataset}, val"] = f'hsv({base_color[0]}, 0.5, 0.9)'
print("-----------------")
for k,v in sorted(color_map.items(),key=lambda x:int(x[0].split(" ")[0])):
    print(k,v)

['1 eclipse_72k, weighted, r=10', '3 mozilla_200k, weighted, r=64', '5 mozilla_200k, weighted, r=64', '7 eclipse_72k, weighted, r=64', '9 eclipse_72k, not_weighted, r=10', '11 mozilla_200k, not_weighted, r=10', '13 eclipse_72k, not_weighted, r=10', '15 mozilla_200k, not_weighted, r=10', '17 eclipse_72k, weighted, r=64', '19 mozilla_200k, weighted, r=64']
-----------------
1 eclipse_72k, weighted, r=10, train hsv(0, 1, 1)
2 eclipse_72k, weighted, r=10, val hsv(0, 0.5, 0.9)
3 mozilla_200k, weighted, r=64, train hsv(216, 1, 1)
4 mozilla_200k, weighted, r=64, val hsv(216, 0.5, 0.9)
5 mozilla_200k, weighted, r=64, train hsv(252, 1, 1)
6 mozilla_200k, weighted, r=64, val hsv(252, 0.5, 0.9)
7 eclipse_72k, weighted, r=64, train hsv(288, 1, 1)
8 eclipse_72k, weighted, r=64, val hsv(288, 0.5, 0.9)
9 eclipse_72k, not_weighted, r=10, train hsv(324, 1, 1)
10 eclipse_72k, not_weighted, r=10, val hsv(324, 0.5, 0.9)
11 mozilla_200k, not_weighted, r=10, train hsv(36, 1, 1)
12 mozilla_200k, not_weighted

In [96]:
print(color_map)
img = px.line(df,x="epoch",y="accuracy_prct",color="id",color_discrete_map=color_map, width=width, height=height, title=title)
img.update_layout(
    legend=dict(traceorder="normal")
)
img.show()
img_to_clipboard(img)

{'1 eclipse_72k, weighted, r=10, train': 'hsv(0, 1, 1)', '2 eclipse_72k, weighted, r=10, val': 'hsv(0, 0.5, 0.9)', '11 mozilla_200k, not_weighted, r=10, train': 'hsv(36, 1, 1)', '12 mozilla_200k, not_weighted, r=10, val': 'hsv(36, 0.5, 0.9)', '13 eclipse_72k, not_weighted, r=10, train': 'hsv(72, 1, 1)', '14 eclipse_72k, not_weighted, r=10, val': 'hsv(72, 0.5, 0.9)', '15 mozilla_200k, not_weighted, r=10, train': 'hsv(108, 1, 1)', '16 mozilla_200k, not_weighted, r=10, val': 'hsv(108, 0.5, 0.9)', '17 eclipse_72k, weighted, r=64, train': 'hsv(144, 1, 1)', '18 eclipse_72k, weighted, r=64, val': 'hsv(144, 0.5, 0.9)', '19 mozilla_200k, weighted, r=64, train': 'hsv(180, 1, 1)', '20 mozilla_200k, weighted, r=64, val': 'hsv(180, 0.5, 0.9)', '3 mozilla_200k, weighted, r=64, train': 'hsv(216, 1, 1)', '4 mozilla_200k, weighted, r=64, val': 'hsv(216, 0.5, 0.9)', '5 mozilla_200k, weighted, r=64, train': 'hsv(252, 1, 1)', '6 mozilla_200k, weighted, r=64, val': 'hsv(252, 0.5, 0.9)', '7 eclipse_72k, wei

In [97]:
img = px.line(df,x="epoch",y="loss",color="id",color_discrete_map=color_map, width=width, height=height, title=title)
img.show()
img_to_clipboard(img)

In [ ]:
lr = [1e-4,1e-5]
qlora_r = [10,64,5]
weighted = [True,False]
qlora_alpha = [4,10]
dropout = [0.1,0,0.2]

print(2*3*2*2*3)